In [ ]:
options(jupyter.rich_display = F)

# LAB 07: MATRICES

## QUESTION 1

For this question, we will use a pre-built dataset which contains the distances in km's between 81 province centers in Turkey.

You first need to download the **distance2.rdata** file from the files directory in Binder and open it. In order to download it, you need to open the files directory, check the box on the left hand side of **distance2.rdata** and click download. When you open the rdata file in your computer you will be able to work with the dataset.

Now write a function named **neighbour** that takes three arguments:

- **city**: Either a city code as integer (i.e. 34) or city name (i.e. "İstanbul")
- **radius**: Radius in km's within which we want to filter the neighbours of the city
- **dist**: A distance matrix, default value of which is the **distance2** global object

as such:

```R
neighbour(34, 150)

  yalova  kocaeli    bursa tekirdag  sakarya  bilecik 
      47       78       91      123      127      128 
```

```R
neighbour(city = "Adana", radius = 100)

  mersin osmaniye 
      64       82 

neighbour("İzmir", 50)

manisa 
    33 
```

*Hint: Be careful not to return the same city as its neighbor.*

## SOLUTION 1

In [ ]:
neighbour <- function(city, radius, dist = distance2)
{
    row <- dist[city,]
    neighs <- row[row <= radius & row > 0]
    return(sort(neighs))
}

neighbour(34, 150)

neighbour(city = "adana", radius = 100)

neighbour("izmir", 50)

## QUESTION 2

Now using the same **distance2** object, write a function named **neigh_count** that takes two arguments:

- **radius** in km's
- a distance matrix called **dist** whose default value is the **distance2** object

The function should return the count of neighbouring cities of each city within the specified radius, sorted in descending order.


Try it with a radius of 100 kilometers:

```R
neigh_count(100)
```

Because of space limitations, we report only the first 3 elements of the output:

```R
bilecik        sakarya        bayburt
      6              5              5
```

*Hint 1: Use the ``apply()`` function.*

*Hint 2: Just like the previous question, number of neighbors must not include the cities themselves.*

## SOLUTION 2

In [ ]:
neigh_count <- function(radius, dist = distance2)
{
    counts <- apply(dist, 1, function(x) sum(x <= radius) - 1)
    return(sort(counts, decreasing = T))
}
                    
neigh_count(100)

## QUESTION 3

You are supposed to write a function named **square_donut** that takes two arguments **n** and **x** and does the following:

- It should create a square matrix of size n.
- The outer x by x squares of the matrix should be filled with 1's, while the square hole in the middle should be all zeros.
- If **x** is greater than or equal to the half of **n**, it should return **"edge too wide"** and stop.

For example:
```R
square_donut(7, 2)
```
returns
```
1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	0	0	0	1	1
1	1	0	0	0	1	1
1	1	0	0	0	1	1
1	1	1	1	1	1	1
1	1	1	1	1	1	1
```
and

```R
square_donut(6, 3)
```
returns
```
'edge too wide'
```

## SOLUTION 3

In [ ]:
square_donut <- function(n, x)
{
    if (x >= n / 2)
    {
        return("edge too wide")
    }
    
    mat1 <- matrix(1, nrow = n, ncol = n) # create matrix of 1's
    startx <- x + 1 # start index of 0's
    endx <- n - x # end index of 0's
    mat1[startx:endx, startx:endx] <- 0 # create the hole of 0's
    
    return(mat1)
}

square_donut(7, 2)
square_donut(6, 3)

## QUESTION 4

First, set the seed to 1.

Now, write a function called **new_field** that takes two inputs, **size** and **density** and creates a square matrix of the given size. The created matrix represents a minefield. The cells should take only two values: if the value is 1, it means there is a mine there and If the value is 0, it means there is no mine. The default values of **size** and **density** should be 5 and 0.2 respectively.

The function should fill the matrix according to **density** which represents the proportion of the cells with mines. When filling the cells, you should use **density** as the probability of any cell containing a mine.

Your function should give the following output:

```R
field1 <- new_field()
field1
```
```
0    1    0    0    1
0    1    0    0    0
0    0    0    1    0
1    0    0    0    0
0    0    0    0    0

```

*Hint: You can use the ``sample()`` function when filling the cells.*

## SOLUTION 4

In [ ]:
set.seed(1)

new_field <- function(size = 5, density = 0.2)
{
    vec <- sample(1:0,
                  size^2,
                  replace = T,
                  prob = c(density, 1 - density))
    mat <- matrix(vec, nrow = size)
    return(mat)
}


field1 <- new_field()
field1

## QUESTION 5

Now write a function called **minefield** that takes 3 arguments:

- **rw**: An integer for the row index
- **cl**: An integer for the column index
- **field**: A matrix for the minefield. Default value is the field1 global object

The function should first retrieve the value of the cell at the rw and cl indices.

If the value is equal to 1, there is a hit, and the function should return 9 (that represents a hit)
Else, there is a miss, then it should return the number of neighbouring hits. In other words, it should return the count of 1's in the neighbouring eight cells of the target.

Some example outputs are given below:

```R
minefield(1,1)
[1] 2

minefield(2,2)
[1] 9

minefield(2,3)
[1] 3
```

## SOLUTION 5

In [ ]:
minefield <- function(rw, cl, field = field1)
{
    val <- field[rw, cl] # get the value
        
    if (val == 1) # there is a hit
    {
        return(9)
    }
    else # there is a miss
    {
        size <- nrow(field) # max index allowed

        rws <- (-1):1 + rw # neighbouring row indices
        rws <- rws[rws > 0 & rws <= size] # filter between 1 and size
        
        cls <- (-1):1 + cl  # neighbouring column indices
        cls <- cls[cls > 0 & cls <= size] # filter between 1 and size
        
        sub <- field[rws, cls] # subset the neighours
        return(sum(sub)) # return the sum of neighbouring hits
    }
}

minefield(1,1)
minefield(2,2)
minefield(2,3)

## QUESTION 6

You are given a matrix generator function as such:

```R
matrix_gen <- function(maxx = 100, nr = 5, nc = 5)
{
    matrix(sample(maxx, nr * nc, replace = T), nrow = nr) 
}
```

Then write a function called **max_path** that will take a matrix **matt** as input and return the maximum sum to be attained when a path is selected from the first row to the end, in each step of which you can select any of the adjacent cells (straight down, down left, down right) on the next row.

First set the seed to 1234.

Now let's try our function with a 3x3 matrix whose cells can take values from 1 to 10. We can construct this matrix using the provided **matrix_gen** function as such:

```R
mat1 <- matrix_gen(10, 3, 3)
mat1
     [,1] [,2] [,3]
[1,]   10    9    4
[2,]    6    5    2
[3,]    5    6    7
```
```R
max_path(mat1)
[1] 22
```

Let's try it one more time (you need to set the seed to 1234 again):

```R
mat2 <- matrix_gen(10, 5, 5)
mat2
     [,1] [,2] [,3] [,4] [,5]
[1,]   10    6   10    4    3
[2,]    6    4    6    5    4
[3,]    5    2    4    8   10
[4,]    9    7    8    4    5
[5,]    5    6    4    8    2
```

```R
max_path(mat2)
[1] 40
```

*Hint: You may make use of ``pmax()`` or ``apply()`` functions.*

## SOLUTION 6

In [ ]:
matrix_gen <- function(maxx = 100, nr = 5, nc = 5)
{
    matrix(sample(maxx, nr * nc, replace = T), nrow = nr)
}

max_path <- function(matt)
{
    # we will track a vector length of which is equal to the number of the rows and
    # that shows the max sums for each cell in the current row up to that row
    # we take the first row as the initial value for this vector (the values show the maximum sum for each column up to that point!)
    sums_vec <- matt[1,]

    # get the dimensions of the original matrix. can also be done with dim()
    nc <- ncol(matt)
    nr <- nrow(matt)
    
    # across rows, starting from the second one
    # on the first row the max values up to that row are the values themselves
    for (roww in 2:nr) 
    {
        # takes three vector:
        # the original sums_vec, the sums_vec offset to left and offset to right
        # this is done by adding a zero to left/trimming from right 
        # or adding a zero to right/trimming from left
        # why? we can come to a cell from straight up, left up or right up
        # so for each cell in the row we should the select the max of these three values
        # pmax does it pairwise (or triple wise in this case)
        # this is the critical part of the code
        max_of_sums_vec <- pmax(c(0, sums_vec[-nc]), sums_vec, c(sums_vec[-1], 0))

        # update the sums vec: add the maximum sums vector (that shows the maximum sums up to that row for each cell)
        # to the current row
        sums_vec <- matt[roww,] + max_of_sums_vec
    }

    # we have come to the last row
    # now just take the maximum of the last sums_vec
    maxval <- max(sums_vec)

    return(maxval)

}

set.seed(1234)
mat1 <- matrix_gen(10, 3, 3)
mat1
max_path(mat1)

set.seed(1234)
mat2 <- matrix_gen(10, 5, 5)
mat2
max_path(mat2)
